In [1]:
import pandas as pd
import os

from library.PY_lib.coordinates import *
from library.PY_lib.read_excel import *
from library.PY_lib.calculate_distance import *
from library.PY_lib.user_define import *
from library.quad_filter import *
from library.PY_lib.flight_scrapper import *
from library.PY_lib.format_json import *
from library.PY_lib.json_to_csv import *
from library.PY_lib.json_to_excel import *
from datetime import datetime
from time import sleep

from openpyxl import Workbook
from openpyxl import load_workbook


# from library.PY_lib.data_retriever import *


In [2]:
ip=input('rentrez G pour générer des données, ou un nom de dossier pour en extraire les données')

In [3]:
if ip != 'G':
    folder = ip
else : folder='input'
# n = automatisation (0 pour oui, 1 pour non)
# r = request left
# t = temps entre chaque requete (en secondes)
n = 1
r = 1
t = 3600 
taille_zone = 100
choice = 2 # 1=quadrilatère, 2 = circulaire
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [4]:
def get_data(n, request_left, attente): #, file_name:str
    file_name = f'{current_time}_flight'
    json_file = f'tempo.json'
    # csv_file = f'tab_{namefile}.csv'
    excel_file = f'{os.path.join("input",file_name)}.xls'
    with open(json_file, 'w') as file:
        pass
    # with open(csv_file, 'w') as file:
    #     pass
    with open(excel_file, 'w') as file:
        pass
    scrap_plane(json_file)
    format_json(json_file, json_file)
    # json_to_csv(json_file, csv_file)
    json_to_excel(json_file, excel_file)
   
    print("all operations done")
    if n == 0 and request_left > 0:
        print(f'waiting {attente} seconds to proceed further, {request_left} request(s) left.')
        request_left = request_left - 1
        sleep(attente)
        get_data(n, request_left, attente)

In [5]:
if ip =='G':
    get_data(n, r - 1, t) #, file_name
    
    

Data written in tempo.json
Data converted to input\2023-10-06_18-18-51_flight.xls
all operations done


In [6]:
file_name = f'{current_time}_flight'
excel_file = f'{os.path.join("input",file_name)}.xls'
df=pd.read_excel(excel_file)
df['Date']=current_time
df.to_excel(excel_file, index=False)

C:\Users\arthu\AppData\Local\Temp\ipykernel_23136\1053186996.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(excel_file, index=False)


In [7]:
def onefiledata(filename, taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude):
    print(filename)
    data = pd.read_excel(filename)
    data = pd.DataFrame(data)

    if choice == 1:
        t_lat = s_latitude
        t_lng = s_longitude
        # filtered_data = []
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            if sort_elements(t_lat[0], t_lat[1], t_lat[2], t_lat[3], lat, t_lng[0], t_lng[1], t_lng[2], t_lng[3], lng) == 0:
                data.loc[index,'distance'] = True
        filtered_data=data.where(data['distance']==True)

    if choice == 2:
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            for ref_lat, ref_lng in zip(r_latitudes, r_longitude):
                data.loc[index,'distance'] = calculate_distance(lat, lng, float(ref_lat), float(ref_lng))
        filtered_data=data.where(data['distance']<=taille_zone)
    # display(filtered_data.dropna())
    return filtered_data.dropna()

In [22]:
round_path = r'D:\Programmes\E.nova\python\PyEnova\flight_track\round_coordinates.txt'
square_path = r'square_coordinates.txt'

r_latitudes, r_longitude = read_coordinates(round_path)
s_latitude, s_longitude = read_coordinates(square_path)

filename = os.listdir(folder)

df=[]
for i in range(len(filename)):
    path2 = os.path.join(folder,filename[i])
    filtered_data=onefiledata(path2,taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude)
    df.append(filtered_data)

if choice==1:
    new_filename = os.path.join('output',f'{current_time}_square_sorted_.xls')
if choice==2: 
    new_filename = os.path.join('output',f'{current_time}_round_sorted_.xls')

filtered_df = pd.concat([*df]) 

filtered_df.to_excel(new_filename, index=False)

print(f"Tout a été enregistré dans le fichier : {new_filename}")

input\2023-10-06_18-15-49_flight.xls
input\2023-10-06_18-18-51_flight.xls
Tout a été enregistré dans le fichier : output\2023-10-06_18-18-51_round_sorted_.xls


C:\Users\arthu\AppData\Local\Temp\ipykernel_23136\340194487.py:22: FutureWarning:

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.



In [42]:
df_p=filtered_df.set_index(['hex', 'Date']).sort_index()

In [100]:
idx=filtered_df['hex'].drop_duplicates(keep=False).index
df_p=filtered_df.drop(index=idx)

In [101]:
df_p.set_index(['hex', 'Date']).sort_index()

reg_number flag        lat       lng      alt  \
hex    Date                                                                
06A1CC 2023-10-06_18-15-49     A7-CGG   QA  43.660904  7.213355      0.0   
       2023-10-06_18-18-51     A7-CGG   QA  43.653259  7.203007      0.0   
39DD44 2023-10-06_18-15-49     F-HXKE   FR  43.204239  6.922409   4572.0   
       2023-10-06_18-18-51     F-HXKE   FR  43.165656  7.226323   4572.0   
3C6669 2023-10-06_18-15-49     D-AISI   DE  43.799927  6.090641  10363.0   
       2023-10-06_18-18-51     D-AISI   DE  44.137253  6.323782  10363.0   
3C674A 2023-10-06_18-15-49     D-AIZJ   DE  43.502884  7.159828    822.0   
       2023-10-06_18-18-51     D-AIZJ   DE  43.631364  7.176165    243.0   
400DB4 2023-10-06_18-15-49     G-EZAS   UK  43.721375  7.020711   5814.0   
       2023-10-06_18-18-51     G-EZAS   UK  43.982834  6.793801   7246.0   
400E4A 2023-10-06_18-15-49     G-GDFJ   UK  43.462234  7.244206   3040.0   
       2023-10-06_18-18-51     G-GDFJ   UK  43.578667  6.964482   4396.0   
440141 2023-10-06_18-15-49     OE-IJX   AT  43.614761  7.311018   1424.0   
       2023-10-06_18-18-51     OE-IJX   AT  43.485535  7.173242   3977.0   
46B821 2023-10-06_18-15-49     SX-NAA   GR  43.329892  7.861829   2575.0   
       2023-10-06_18-18-51     SX-NAA   GR  43.352829  7.546585   1303.0   
4D2104 2023-10-06_18-15-49     9H-BOO   MT  43.447357  6.508633   3268.0   
       2023-10-06_18-18-51     9H-BOO   MT  43.402725  6.822318   2125.0   
4D22B1 2023-10-06_18-15-49     9H-LOZ   IE  43.358289  6.893594  10972.0   
       2023-10-06_18-18-51     9H-LOZ   IE  43.581253  7.306675  10972.0   

                              dir  speed  v_speed  squawk flight_number  ...  \
hex    Date                                                              ...   
06A1CC 2023-10-06_18-15-49  225.0   13.0      0.0  1000.0           511  ...   
       2023-10-06_18-18-51  143.0   13.0      0.0  1000.0           511  ...   
39DD44 2023-10-06_18-15-49   99.0  507.0      0.0  1000.0          41QA  ...   
       2023-10-06_18-18-51  100.0  509.0      0.0  1000.0          41QA  ...   
3C6669 2023-10-06_18-15-49   26.0  853.0      0.3  1057.0           12N  ...   
       2023-10-06_18-18-51   26.0  850.0      0.0  1057.0           12N  ...   
3C674A 2023-10-06_18-15-49  356.4  358.0      0.0  1000.0           55J  ...   
       2023-10-06_18-18-51   45.0  244.0     -3.3  1000.0           55J  ...   
400DB4 2023-10-06_18-15-49  328.1  627.0     13.3  6753.0          48WT  ...   
       2023-10-06_18-18-51  327.8  746.0      8.8  6753.0          48WT  ...   
400E4A 2023-10-06_18-15-49  275.9  452.0      8.5  4070.0          62VC  ...   
       2023-10-06_18-18-51  322.2  625.0     11.1  4070.0          62VC  ...   
440141 2023-10-06_18-15-49  168.1  414.0      5.5  1000.0          908G  ...   
       2023-10-06_18-18-51  279.6  512.0      7.5  1000.0          908G  ...   
46B821 2023-10-06_18-15-49  276.0  533.0     -7.5  2046.0           690  ...   
       2023-10-06_18-18-51  276.0  480.0     -3.3  2046.0           690  ...   
4D2104 2023-10-06_18-15-49  102.0  555.0     -8.1  3322.0          629P  ...   
       2023-10-06_18-18-51  101.1  488.0     -5.2  3322.0          629P  ...   
4D22B1 2023-10-06_18-15-49   53.0  844.0      0.0  5542.0          33ZT  ...   
       2023-10-06_18-18-51   53.0  840.0     -0.4  5542.0          33ZT  ...   

                           dep_icao dep_iata arr_icao arr_iata airline_icao  \
hex    Date                                                                   
06A1CC 2023-10-06_18-15-49     LFMN      NCE     LFBD      BOD          QQE   
       2023-10-06_18-18-51     LFMN      NCE     LFBD      BOD          QQE   
39DD44 2023-10-06_18-15-49     LFML      MRS     LFKC      CLY          CCM   
       2023-10-06_18-18-51     LFML      MRS     LFKC      CLY          CCM   
3C6669 2023-10-06_18-15-49     LEBL      BCN     EDDF      FRA          DLH   
       2023-10-06_18-18-51     LEBL     

In [ ]:
import plotly_express as px

In [ ]:
fig=px.scatter_geo(
    data_frame=filtered_df,
    lat='lat',
    lon='lng',
    hover_name='reg_number',
    color='flag',
    hover_data=['alt', 'speed'],
    # text=,
    )
fig.write_html(os.path.join('output',f'{current_time}_Flight_map'))


In [ ]:
fig

In [ ]:
fig = px.scatter(
    data_frame=filtered_df,
    y='alt',
    x='speed',
)
fig.show()